# 模型保存与加载

### 序列化与反序列化
```
模型在内存中是以对象的逻辑结构保存的，但是在硬盘中是以二进制流的方式保存的。
    序列化是指将内存中的数据以二进制序列的方式保存到硬盘中。PyTorch 的模型保存就是序列化。
    反序列化是指将硬盘中的二进制序列加载到内存中，得到模型的对象。PyTorch 的模型加载就是反序列化。

### PyTorch 中的模型保存与加载
#### torch.save
```python
torch.save(obj, f, pickle_module, pickle_protocol=2, _use_new_zipfile_serialization=False)
```
```
主要参数：
    obj：保存的对象，可以是模型。也可以是 dict。因为一般在保存模型时，不仅要保存模型，还需要保存优化器、此时对应的 epoch 等参数。这时就可以用 dict 包装起来。 
    f：输出路径

#### 保存整个 Module
```
这种方法比较耗时，保存的文件大
```
```python
torch.savev(net, path)
```

#### 只保存模型的参数
```
推荐这种方法，运行比较快，保存的文件比较小
```
```python
state_sict = net.state_dict()
torch.savev(state_sict, path)

In [6]:
import torch
import numpy as np
import torch.nn as nn



class LeNet2(nn.Module):
    def __init__(self, classes):
        super(LeNet2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

    def initialize(self):
        for p in self.parameters():
            p.data.fill_(2020)


net = LeNet2(classes=2019)

# "训练"
print("训练前: ", net.features[0].weight[0, ...])

net.initialize()

print("训练后: ", net.features[0].weight[0, ...])

path_model = "./model.pkl"
path_state_dict = "./model_state_dict.pkl"

# 保存整个模型
torch.save(net, path_model)

# 保存模型参数
net_state_dict = net.state_dict()
torch.save(net_state_dict, path_state_dict)


# 运行完之后，文件夹中生成了`model.pkl和model_state_dict.pkl，分别保存了整个网络和网络的参数

训练前:  tensor([[[ 0.0876,  0.0529, -0.0758, -0.1075, -0.0022],
         [ 0.0800, -0.0787,  0.0389,  0.0331, -0.0408],
         [-0.1009, -0.1117,  0.0450, -0.0704, -0.0501],
         [-0.0836,  0.0471, -0.0057,  0.1027, -0.0473],
         [ 0.0197, -0.0337, -0.0935,  0.0784, -0.0471]],

        [[ 0.0742,  0.0455,  0.0761, -0.0372, -0.0105],
         [ 0.1002, -0.1115, -0.0587,  0.1089, -0.0836],
         [-0.0340,  0.0857, -0.1010, -0.0612,  0.0431],
         [-0.0432,  0.1027,  0.0263,  0.0898, -0.1027],
         [-0.0713,  0.1027,  0.0258, -0.0265,  0.0360]],

        [[ 0.0162, -0.0630, -0.0137, -0.0632, -0.0607],
         [ 0.0404, -0.1099, -0.0950,  0.0584, -0.0649],
         [ 0.0251, -0.1137,  0.0732,  0.0922, -0.1001],
         [ 0.0896,  0.1099, -0.0756, -0.0201, -0.0048],
         [ 0.0420,  0.1042, -0.0145,  0.0009,  0.0347]]],
       grad_fn=<SelectBackward0>)
训练后:  tensor([[[2020., 2020., 2020., 2020., 2020.],
         [2020., 2020., 2020., 2020., 2020.],
         [2020.,

#### torch.load
```python
torch.load(f, map_location=None, pickle_module, **pickle_load_args)
```
```
主要参数：
    f：文件路径
    map_location：指定存在 CPU 或者 GPU。

#### 加载整个 Module
```
如果保存的时候，保存的是整个模型，那么加载时就加载整个模型。这种方法不需要事先创建一个模型对象，也不用知道模型的结构，代码如下：

In [7]:
path_model = "./model.pkl"
net_load = torch.load(path_model)

print(net_load)

LeNet2(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=2019, bias=True)
  )
)


#### 只加载模型的参数
```
如果保存的时候，保存的是模型的参数，那么加载时就参数。这种方法需要事先创建一个模型对象，再使用模型的load_state_dict()方法把参数加载到模型中，代码如下：

In [8]:
path_state_dict = "./model_state_dict.pkl"
state_dict_load = torch.load(path_state_dict)

net_new = LeNet2(classes=2019)

print("加载前: ", net_new.features[0].weight[0, ...])

net_new.load_state_dict(state_dict_load)

print("加载后: ", net_new.features[0].weight[0, ...])

加载前:  tensor([[[ 0.0165, -0.0856, -0.1038, -0.0061,  0.0068],
         [-0.0089, -0.0669,  0.0121, -0.0423,  0.0216],
         [ 0.0420, -0.1152, -0.0966, -0.0857,  0.0964],
         [ 0.0601, -0.0246,  0.0148,  0.0036, -0.0428],
         [-0.0912,  0.0530,  0.0408, -0.0520,  0.0635]],

        [[-0.0100, -0.0641, -0.1150,  0.1150, -0.1132],
         [-0.0359,  0.0303,  0.0443, -0.0682, -0.1075],
         [-0.0633,  0.0084,  0.0239,  0.0352, -0.0664],
         [ 0.0189,  0.0043, -0.1002,  0.1127,  0.0960],
         [-0.0592, -0.0249,  0.0394, -0.0787, -0.0479]],

        [[ 0.1062,  0.1113,  0.0452, -0.0841, -0.0681],
         [-0.0952, -0.0220,  0.1072,  0.1151,  0.0272],
         [ 0.0846, -0.1028,  0.0993,  0.0293,  0.0557],
         [-0.0477, -0.0834,  0.0692, -0.0096,  0.0268],
         [-0.0450,  0.0685,  0.0991, -0.0224,  0.1033]]],
       grad_fn=<SelectBackward0>)
加载后:  tensor([[[2020., 2020., 2020., 2020., 2020.],
         [2020., 2020., 2020., 2020., 2020.],
         [2020.,

### 模型的断点续训练
```
在训练过程中，可能由于某种意外原因如断点等导致训练终止，这时需要重新开始训练。断点续练是在训练过程中每隔一定次数的 epoch 就保存模型的参数和优化器的参数，这样如果意外终止训练了，下次就可以重新加载最新的模型参数和优化器的参数，在这个基础上继续训练。

下面的代码中，每隔 5 个 epoch 就保存一次，保存的是一个 dict，包括模型参数、优化器的参数、epoch。然后在 epoch 大于 5 时，就break模拟训练意外终止。关键代码如下：
```
```python
    if (epoch+1) % checkpoint_interval == 0:

        checkpoint = {"model_state_dict": net.state_dict(),
                      "optimizer_state_dict": optimizer.state_dict(),
                      "epoch": epoch}
        path_checkpoint = "./checkpoint_{}_epoch.pkl".format(epoch)
        torch.save(checkpoint, path_checkpoint)
```
```
在 epoch 大于 5 时，就break模拟训练意外终止
```
```python
    if epoch > 5:
        print("训练意外中断...")
        break
```
```
断点续训练的恢复代码如下：
```
```python
path_checkpoint = "./checkpoint_4_epoch.pkl"
checkpoint = torch.load(path_checkpoint)

net.load_state_dict(checkpoint['model_state_dict'])

optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

start_epoch = checkpoint['epoch']

scheduler.last_epoch = start_epoch

# 需要注意的是，还要设置scheduler.last_epoch参数为保存的 epoch。模型训练的起始 epoch 也要修改为保存的 epoch。